Import Data

In [1]:
import pandas as pd
df = pd.read_csv('turnstile_170617.txt', parse_dates = [['DATE', 'TIME']])
df.columns = df.columns.str.strip()
df = df.rename(columns = {'ENTRIES': 'CUM_ENTRIES', 'EXITS': 'CUM_EXITS'})
df = df.dropna()
df.head()

,DATE_TIME,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DESC,CUM_ENTRIES,CUM_EXITS
0,2017-06-10 00:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6215258,2104297
1,2017-06-10 04:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6215284,2104303
2,2017-06-10 08:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6215318,2104337
3,2017-06-10 12:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6215475,2104417
4,2017-06-10 16:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6215841,2104465


In [2]:
df_copy = df.copy()

In [3]:
#we care about
#C/A + UNIT + STATION refers to a unique turnstile
#SCP denotes the nearby(next to the turnstile) train locations
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 197209 entries, 0 to 197208
Data columns (total 10 columns):
DATE_TIME      197209 non-null datetime64[ns]
C/A            197209 non-null object
UNIT           197209 non-null object
SCP            197209 non-null object
STATION        197209 non-null object
LINENAME       197209 non-null object
DIVISION       197209 non-null object
DESC           197209 non-null object
CUM_ENTRIES    197209 non-null int64
CUM_EXITS      197209 non-null int64
dtypes: datetime64[ns](1), int64(2), object(7)
memory usage: 11.3+ MB


Create Groupby Object (Each object in the list represents a turnstile)

In [4]:
list_of_df = [g for _, g in df_copy.groupby(['C/A', 'SCP', 'UNIT','STATION'])]
keys = [_ for _,g in df_copy.groupby(['C/A', 'SCP', 'UNIT','STATION'])]
for i, g in enumerate(list_of_df):
    list_of_df[i].sort_values(by="DATE_TIME", inplace=True)

In [5]:
list_of_df[0].head()

,DATE_TIME,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DESC,CUM_ENTRIES,CUM_EXITS
0,2017-06-10 00:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6215258,2104297
1,2017-06-10 04:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6215284,2104303
2,2017-06-10 08:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6215318,2104337
3,2017-06-10 12:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6215475,2104417
4,2017-06-10 16:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6215841,2104465


Adding Amount of People Exiting the Station

In [6]:
for i, x in enumerate(list_of_df):
    ENTRIES = [0]
    EXITS = [0]
    for j in range(len(x.index)):
        if j==0:
            continue
        now = x.iloc[j]
        prev = x.iloc[j-1]
        EXITS.append(now.CUM_EXITS-prev.CUM_EXITS)
        ENTRIES.append(now.CUM_ENTRIES-prev.CUM_ENTRIES)
    list_of_df[i]['ENTRIES'] = ENTRIES
    list_of_df[i]['EXITS'] = EXITS
    list_of_df[i]["PEOPLE"] = list_of_df[i].ENTRIES+list_of_df[i].EXITS
    list_of_df[i]["DATE"] = [d.date() for d in list_of_df[i].DATE_TIME]
    list_of_df[i]["TIME"] = [d.time() for d in list_of_df[i].DATE_TIME]

In [7]:
list_of_df[0].head()

,DATE_TIME,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DESC,CUM_ENTRIES,CUM_EXITS,ENTRIES,EXITS,PEOPLE,DATE,TIME
0,2017-06-10 00:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6215258,2104297,0,0,0,2017-06-10,00:00:00
1,2017-06-10 04:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6215284,2104303,26,6,32,2017-06-10,04:00:00
2,2017-06-10 08:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6215318,2104337,34,34,68,2017-06-10,08:00:00
3,2017-06-10 12:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6215475,2104417,157,80,237,2017-06-10,12:00:00
4,2017-06-10 16:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6215841,2104465,366,48,414,2017-06-10,16:00:00


This is what the data should look like

In [8]:
import matplotlib.pyplot as plt
%matplotlib inline
x = list_of_df[0].DATE_TIME
y = list_of_df[0].CUM_EXITS
plt.plot(x,y)
plt.xticks([])
plt.show()

Extracting all errors

In [10]:
errors = []
for g in list_of_df:
    temp = g[(g.ENTRIES > 10000) | (g.EXITS > 10000) | (g.ENTRIES<0) | (g.EXITS < 0) | ((g.ENTRIES/g.CUM_ENTRIES>0.5)&(g.CUM_ENTRIES>20000)) | ((g.EXITS/g.CUM_EXITS>0.5) & (g.CUM_EXITS>20000))]
    if not temp.empty:
        errors.append(g)
len(errors)

84

In [11]:
import matplotlib.pyplot as plt
%matplotlib inline
for x in errors:
    plt.plot(x.DATE_TIME, x.CUM_EXITS)
    plt.xticks([])
    plt.show()

In [12]:
reflected_entries = []
for x in errors:
    if len(x[x.ENTRIES < 0]) == len(x)-1:
        reflected_entries.append(x)

for x in reflected_entries:
    plt.plot(x.DATE_TIME, x.CUM_ENTRIES, '.')
    plt.xticks([])
    plt.show()

In [15]:
for i, x in enumerate(reflected_entries):
    CUM_ENTRIES = []
    ENTRIES = []
    for j in range(len(x.index)):
        if j==0:
            CUM_ENTRIES.append(x.iloc[j].CUM_ENTRIES)
            ENTRIES.append(0)
            continue
        prev = x.iloc[j-1]
        now = x.iloc[j]
        ENTRIES.append(-now.ENTRIES)
        CUM_ENTRIES.append(CUM_ENTRIES[-1]+ENTRIES[-1])
    reflected_entries[i].CUM_ENTRIES = CUM_ENTRIES
    reflected_entries[i].ENTRIES = ENTRIES
    reflected_entries[i].PEOPLE = reflected_entries[i].ENTRIES+reflected_entries[i].EXITS
for x in reflected_entries:
    plt.plot(x.DATE_TIME, x.CUM_ENTRIES, '.')
    plt.xticks([])
    plt.show()

In [16]:
reflected_exits = []
for x in errors:
    if len(x[x.EXITS < 0]) == len(x)-1:
        reflected_exits.append(x)

for x in reflected_exits:
    plt.plot(x.DATE_TIME, x.CUM_EXITS, '.')
    plt.xticks([])
    plt.show()

In [17]:
for i, x in enumerate(reflected_exits):
    CUM_EXITS = []
    EXITS = []
    for j in range(len(x.index)):
        if j==0:
            CUM_EXITS.append(x.iloc[j].CUM_EXITS)
            EXITS.append(0)
            continue
        prev = x.iloc[j-1]
        now = x.iloc[j]
        EXITS.append(-now.EXITS)
        CUM_EXITS.append(CUM_EXITS[-1]+EXITS[-1])
    reflected_exits[i].CUM_EXITS = CUM_EXITS
    reflected_exits[i].EXITS = EXITS
    reflected_exits[i].PEOPLE = reflected_exits[i].ENTRIES+reflected_exits[i].EXITS
for x in reflected_exits:
    plt.plot(x.DATE_TIME, x.CUM_EXITS, '.')
    plt.xticks([])
    plt.show()

Cleanup and get no error dataframe

In [18]:
noerrors = []
for i, x in enumerate(list_of_df):
    noerrors.append(x[(x.ENTRIES>0) & (x.EXITS>0) &((x.ENTRIES/x.CUM_ENTRIES<=0.5) | x.CUM_ENTRIES<=20000) & ((x.EXITS/x.CUM_EXITS<=0.5) | (x.CUM_EXITS<=20000))])
result = pd.concat(noerrors)

In [21]:
# result = result.drop(['CUM_EXITS','CUM_ENTRIES'],axis=1)
result.sort_values(by="PEOPLE",ascending=False).head()

,DATE_TIME,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DESC,ENTRIES,EXITS,PEOPLE,DATE,TIME
152270,2017-06-14 12:00:00,R238,R046,00-06-02,GRD CNTRL-42 ST,4567S,IRT,REGULAR,650,3909,4559,2017-06-14,12:00:00
152276,2017-06-15 12:00:00,R238,R046,00-06-02,GRD CNTRL-42 ST,4567S,IRT,REGULAR,653,3897,4550,2017-06-15,12:00:00
49047,2017-06-13 20:00:00,N063A,R011,00-00-00,42 ST-PORT AUTH,ACENQRS1237W,IND,REGULAR,81,4398,4479,2017-06-13,20:00:00
152264,2017-06-13 12:00:00,R238,R046,00-06-02,GRD CNTRL-42 ST,4567S,IRT,REGULAR,477,3941,4418,2017-06-13,12:00:00
151971,2017-06-14 20:00:00,R238,R046,00-03-02,GRD CNTRL-42 ST,4567S,IRT,REGULAR,4184,200,4384,2017-06-14,20:00:00


In [22]:
import pickle
result.to_pickle("No_Errors_Dataset.pkl")

In [4]:
import pandas as pd
result = pd.read_pickle("No_Errors_Dataset.pkl")
result.sort_values(by="PEOPLE",ascending=False).head()

,DATE_TIME,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DESC,ENTRIES,EXITS,PEOPLE,DATE,TIME
152270,2017-06-14 12:00:00,R238,R046,00-06-02,GRD CNTRL-42 ST,4567S,IRT,REGULAR,650,3909,4559,2017-06-14,12:00:00
152276,2017-06-15 12:00:00,R238,R046,00-06-02,GRD CNTRL-42 ST,4567S,IRT,REGULAR,653,3897,4550,2017-06-15,12:00:00
49047,2017-06-13 20:00:00,N063A,R011,00-00-00,42 ST-PORT AUTH,ACENQRS1237W,IND,REGULAR,81,4398,4479,2017-06-13,20:00:00
152264,2017-06-13 12:00:00,R238,R046,00-06-02,GRD CNTRL-42 ST,4567S,IRT,REGULAR,477,3941,4418,2017-06-13,12:00:00
151971,2017-06-14 20:00:00,R238,R046,00-03-02,GRD CNTRL-42 ST,4567S,IRT,REGULAR,4184,200,4384,2017-06-14,20:00:00


Generate dictionary containing all the data

In [5]:
alldict = {}
stations = [(_,g) for _,g in result.groupby(["C/A","UNIT","STATION"])]
for k,x in stations:
    scp = [(_,g) for _,g in x.groupby(["SCP"])]
    alldict[k] = {}
    for y,t in scp:
        alldict[k][y] = []
        time_list = t.DATE_TIME.tolist()
        people_list = t.PEOPLE.tolist()
        for i in range(len(time_list)):
            alldict[k][y].append((time_list[i],people_list[i]))
        alldict[k][y].sort(key=lambda x:x[0])
# scp[0][1]
alldict

Find busiest 4 hrs interval during each day for each station

In [6]:
import datetime
def getPeopleInterval(data, start, end):
    people = 0
    for x in data:
        if start<=x[0]<=end:
            people+=x[1]
    return people
days_range = pd.date_range('2017-06-10',periods=7)
busiest = {}
for key,val in alldict.items():
    busiest[key] = {}
    for x in days_range:
        time_range = pd.date_range(start=x,end=x+datetime.timedelta(hours=20),freq='1H')
        temp = []
        for start in time_range:
            people = 0
            for scp,data in val.items():
                people += getPeopleInterval(data, start, start+datetime.timedelta(hours=4))
            temp.append((people, start))
        busiest[key][x] = sorted(temp,key=lambda x:-x[0])[0]
#         print(temp)

Find busiest stations for each day

In [7]:
station_dict = {}
for _,g in result.groupby(["C/A","UNIT","STATION"]):
    station_dict[_] = pd.DataFrame({'PEOPLE':g.groupby("DATE").PEOPLE.sum()})
    station_dict[_] = station_dict[_].reset_index()
station_dict[('A002', 'R051', '59 ST')]

,DATE,PEOPLE
0,2017-06-10,10396
1,2017-06-11,10971
2,2017-06-12,20080
3,2017-06-13,22308
4,2017-06-14,23853
5,2017-06-15,24062
6,2017-06-16,23561


In [8]:
days = station_dict[('A002', 'R051', '59 ST')]["DATE"].tolist()
each_day = []
for x in days:
    each_day.append([])
    for key,val in station_dict.items():
        if val[val["DATE"] == x].empty:
            continue
        each_day[-1].append((key, int(val[val["DATE"]==x].PEOPLE)))
for i, x in enumerate(each_day):
    each_day[i].sort(key=lambda x:-x[1])
    print(each_day[i][:5])

[(('H009', 'R235', 'BEDFORD AV'), 60898), (('R238', 'R046', 'GRD CNTRL-42 ST'), 48734), (('N324', 'R018', 'JKSN HT-ROOSVLT'), 48451), (('R533', 'R055', 'FLUSHING-MAIN'), 46575), (('R138', 'R293', '34 ST-PENN STA'), 43244)]
[(('H009', 'R235', 'BEDFORD AV'), 57219), (('R238', 'R046', 'GRD CNTRL-42 ST'), 52248), (('N324', 'R018', 'JKSN HT-ROOSVLT'), 46791), (('R533', 'R055', 'FLUSHING-MAIN'), 46705), (('N506', 'R022', '34 ST-HERALD SQ'), 44889)]
[(('PTH22', 'R540', 'PATH NEW WTC'), 113509), (('R238', 'R046', 'GRD CNTRL-42 ST'), 106033), (('R240', 'R047', 'GRD CNTRL-42 ST'), 80892), (('R138', 'R293', '34 ST-PENN STA'), 74494), (('R533', 'R055', 'FLUSHING-MAIN'), 70689)]
[(('PTH22', 'R540', 'PATH NEW WTC'), 115641), (('R238', 'R046', 'GRD CNTRL-42 ST'), 111142), (('R240', 'R047', 'GRD CNTRL-42 ST'), 86889), (('R138', 'R293', '34 ST-PENN STA'), 75637), (('R533', 'R055', 'FLUSHING-MAIN'), 71004)]
[(('PTH22', 'R540', 'PATH NEW WTC'), 113584), (('R238', 'R046', 'GRD CNTRL-42 ST'), 103919), (('R

In [45]:
days = pd.date_range('2017-06-10', periods = 7)
hours = [0] * 24
stations = {} 
for i,x in enumerate(each_day):
    print("Day: %d"%(i+1))
    for station,people in x[:10]:
        print(station, busiest[station][days[i]])
        start = busiest[station][days[i]][1].hour
        sta = station[2]
        if sta not in stations:
            stations[sta] = 1
        else:
            stations[sta] = stations[sta] + 1
        for j in range(4):   
            hours[start + j - 1] = hours[start+ j - 1] + 1

Day: 1
('H009', 'R235', 'BEDFORD AV') (45698, Timestamp('2017-06-10 16:00:00', freq='H'))
('R238', 'R046', 'GRD CNTRL-42 ST') (32201, Timestamp('2017-06-10 16:00:00', freq='H'))
('N324', 'R018', 'JKSN HT-ROOSVLT') (26656, Timestamp('2017-06-10 16:00:00', freq='H'))
('R533', 'R055', 'FLUSHING-MAIN') (28074, Timestamp('2017-06-10 16:00:00', freq='H'))
('R138', 'R293', '34 ST-PENN STA') (25444, Timestamp('2017-06-10 18:00:00', freq='H'))
('N506', 'R022', '34 ST-HERALD SQ') (28250, Timestamp('2017-06-10 16:00:00', freq='H'))
('N083', 'R138', 'W 4 ST-WASH SQ') (25225, Timestamp('2017-06-10 17:00:00', freq='H'))
('PTH22', 'R540', 'PATH NEW WTC') (12777, Timestamp('2017-06-10 17:00:00', freq='H'))
('R151', 'R033', 'TIMES SQ-42 ST') (26304, Timestamp('2017-06-10 20:00:00', freq='H'))
('A034', 'R170', '14 ST-UNION SQ') (24134, Timestamp('2017-06-10 17:00:00', freq='H'))
Day: 2
('H009', 'R235', 'BEDFORD AV') (30548, Timestamp('2017-06-11 16:00:00', freq='H'))
('R238', 'R046', 'GRD CNTRL-42 ST') 

In [40]:
import matplotlib.pyplot as plt
import numpy as np
indices = np.arange(len(hours))
plt.bar(indices, hours, color='c')
plt.xticks(indices, rotation='vertical')
plt.tight_layout()
plt.title("Volunteer Service Hours")
plt.xlabel('Time (h))')
plt.ylabel('Cummulative times of volunteering')

plt.show()


{'BEDFORD AV': 5, 'GRD CNTRL-42 ST': 12, 'JKSN HT-ROOSVLT': 7, 'FLUSHING-MAIN': 7, '34 ST-PENN STA': 7, '34 ST-HERALD SQ': 8, 'W 4 ST-WASH SQ': 2, 'PATH NEW WTC': 7, 'TIMES SQ-42 ST': 2, '14 ST-UNION SQ': 7, '42 ST-PORT AUTH': 4, '47-50 STS ROCK': 1, '59 ST COLUMBUS': 1}
['BEDFORD AV', 'GRD CNTRL-42 ST', 'JKSN HT-ROOSVLT', 'FLUSHING-MAIN', '34 ST-PENN STA', '34 ST-HERALD SQ', 'W 4 ST-WASH SQ', 'PATH NEW WTC', 'TIMES SQ-42 ST', '14 ST-UNION SQ', '42 ST-PORT AUTH', '47-50 STS ROCK', '59 ST COLUMBUS']


In [66]:
df = pd.DataFrame([stations], columns=stations.keys())

In [64]:
df

,BEDFORD AV,GRD CNTRL-42 ST,JKSN HT-ROOSVLT,FLUSHING-MAIN,34 ST-PENN STA,34 ST-HERALD SQ,W 4 ST-WASH SQ,PATH NEW WTC,TIMES SQ-42 ST,14 ST-UNION SQ,42 ST-PORT AUTH,47-50 STS ROCK,59 ST COLUMBUS
0,5,12,7,7,7,8,2,7,2,7,4,1,1
